In [1]:
import pandas as pd
import math as m
import numpy as np
from timeit import default_timer as timer

In [2]:
df = pd.read_csv('total_df_coin_2018_03_01.csv', sep=',')

In [3]:
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [4]:
for i in range(0, len(df)):
    if isfloat(df.loc[i, 'Open']) == True:
        pass
    else:
        df.loc[i, 'Open'] = df['Open'][i].replace(',', '')
#     df.loc[i, 'Open'] = float(df.loc[i, 'Open'])

In [5]:
for i in range(0, len(df)):
    df.loc[i, 'Open'] = float(df.loc[i, 'Open'])

In [6]:
all_share_in_table = df['ticker'].unique().tolist()

In [7]:
all_share_in_table

['BCH',
 'BTC',
 'ETH',
 'LTC',
 'XRP',
 'DASH',
 'ETC',
 'XMR',
 'BTC_ETH',
 'BTC_ETC',
 'ETH_ETC']

In [8]:
len(df)

9714

In [9]:
df.head(10)

,Date,Price,Open,High,Low,Vol.,Change %,ticker
0,2017-08-03,327.40,399,435.00,302.10,46.98K,-17.94,BCH
1,2017-08-04,220.61,327.4,354.00,220.50,88.94K,-32.62,BCH
2,2017-08-05,210.00,220.61,277.45,194.66,82.67K,-4.81,BCH
3,2017-08-06,217.75,210,222.99,190.10,73.03K,3.69,BCH
4,2017-08-07,320.00,217.75,370.00,215.00,233.67K,46.96,BCH
5,2017-08-08,342.23,320,387.20,307.50,147.21K,6.95,BCH
6,2017-08-09,301.00,342.23,348.00,287.00,71.42K,-12.05,BCH
7,2017-08-10,273.11,301,312.00,265.00,63.55K,-9.27,BCH
8,2017-08-11,325.54,273.11,351.47,270.01,82.96K,19.2,BCH
9,2017-08-12,315.09,325.54,345.25,308.00,29.02K,-3.21,BCH


In [10]:
df = df[df.Date >= '2015-01-01']
df = df.reset_index()
df = df.drop('index', axis=1)
df.head()

,Date,Price,Open,High,Low,Vol.,Change %,ticker
0,2017-08-03,327.40,399,435.00,302.10,46.98K,-17.94,BCH
1,2017-08-04,220.61,327.4,354.00,220.50,88.94K,-32.62,BCH
2,2017-08-05,210.00,220.61,277.45,194.66,82.67K,-4.81,BCH
3,2017-08-06,217.75,210,222.99,190.10,73.03K,3.69,BCH
4,2017-08-07,320.00,217.75,370.00,215.00,233.67K,46.96,BCH


In [11]:
len(df)

8652

In [12]:
df['p_sell'] = 0
df['p_buy'] = 0
df['day_profit'] = 0
df['total_profit'] = 0
df['sell/buy'] = 0
df['count_sell'] = 0
df['count_buy'] = 0
df['count_total_buy'] = 0
df['costs_of_bying'] = 0
df['sum_invested'] = 0

# column_p_sell = 'p_sell'
# column_p_buy = 'p_buy'
# column_price = 'Open'
# column_sell_buy = 'sell/buy'
# column_day_profit = 'day_profit'
# column_profit = 'total_profit'
# column_count_sell = 'count_sell'
# column_count_buy = 'count_buy'
# column_count_total_buy = 'count_total_buy'
# column_costs_of_bying = 'costs_of_bying'
# column_sum_invested = 'sum_invested'
# column_ticker = 'ticker'

In [13]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %,ticker,p_sell,p_buy,day_profit,total_profit,sell/buy,count_sell,count_buy,count_total_buy,costs_of_bying,sum_invested
0,2017-08-03,327.40,399,435.00,302.10,46.98K,-17.94,BCH,0,0,0,0,0,0,0,0,0,0
1,2017-08-04,220.61,327.4,354.00,220.50,88.94K,-32.62,BCH,0,0,0,0,0,0,0,0,0,0
2,2017-08-05,210.00,220.61,277.45,194.66,82.67K,-4.81,BCH,0,0,0,0,0,0,0,0,0,0
3,2017-08-06,217.75,210,222.99,190.10,73.03K,3.69,BCH,0,0,0,0,0,0,0,0,0,0
4,2017-08-07,320.00,217.75,370.00,215.00,233.67K,46.96,BCH,0,0,0,0,0,0,0,0,0,0


In [14]:
list_indexs = []
share_dict_start_end = dict()
for j in range(0, len(all_share_in_table)):    
    share_dict_start_end[all_share_in_table[j]] = 0
    for i in range(0, len(df)):
        if df.loc[i, 'ticker'] == all_share_in_table[j]:
            list_indexs.append(i)            
    w = list_indexs[0]
    w_last = list_indexs[-1]
    share_dict_start_end[all_share_in_table[j]] = [w, w_last]
    list_indexs = []

In [15]:
share_dict_start_end

{'BCH': [0, 208],
 'BTC': [209, 1347],
 'ETH': [1348, 2067],
 'LTC': [2068, 2620],
 'XRP': [2621, 3744],
 'DASH': [3745, 4850],
 'ETC': [4851, 5429],
 'XMR': [5430, 6554],
 'BTC_ETH': [6555, 7488],
 'BTC_ETC': [7489, 8071],
 'ETH_ETC': [8072, 8651]}

In [16]:
def prod(j, array):
    p = 1
    if j == 0:
        return p * (1 - array[0])
    else:
        return prod(j-1, array) * (1-array[j])

In [17]:
procent = [0, 0.15, 0.20, 0.25, 0.30]
amounts_S = [1000,1000,2000,4000,8000]
r_fin = 10
procent_loss = 10
r = 10
count_step = [0]*(len(amounts_S) + 1)
size_profit = [0]*(len(amounts_S) + 1)
count_days = [0]*(len(amounts_S) + 1)

In [18]:
prod(4, procent)

0.357

In [19]:
def model_5k(df, list_share_start, list_share_end):
    w = list_share_start
    p0 = df.loc[w, column_price]
    
# визначаємо яку кількість акцій потрібно купувати на відповідному етапі докуповування
    number = []
    for j in range(0, len(amounts_S)):
        if j==0:
            number.append(amounts_S[j]/p0)
        else:
            number.append(amounts_S[j]/(p0*prod(j, procent)))

    k0 = number[0]  
    K = k0
    S0 = k0 * p0
    C = S0
    Profit = 0
    t = 0
    p_sell = p0 * (1 + r/100)
    p_buy = p0 * (1 - procent[1])

    df.loc[w, column_p_sell] = p_sell
    df.loc[w, column_p_buy] = p_buy
    df.loc[w, column_sell_buy] = 'buy'
    df.loc[w, column_day_profit] = K * df.loc[w, column_price] - C
    df.loc[w, column_profit] = Profit
    df.loc[w, column_count_buy] = k0
    df.loc[w, column_count_total_buy] = K
    df.loc[w, column_costs_of_bying] = S0
    df.loc[w, column_sum_invested] = C

    for i in range(w + 1, list_share_end + 1): 
        count_days[t] = count_days[t] + 1
        count_days
            
        if df[column_price][i] > p_sell:
            if t < len(amounts_S):
                count_step[t] = count_step[t] + 1
                count_step
                size_profit[t] = size_profit[t] + K * df[column_price][i] - C
                size_profit
            else:
                pass
            Profit = Profit + ( K*df[column_price][i] - C)
            df.loc[i, column_profit] = Profit
            df.loc[i, column_count_sell] = K
            
            p0 = df[column_price][i]
            p_sell = p0*(1 + r/100)
            p_buy = p0*(1 - procent[1])

            df.loc[i, column_p_sell] = p_sell
            df.loc[i, column_p_buy] = p_buy
            df.loc[i, column_sell_buy] = 'sell'
            df.loc[i, column_day_profit] = K * df.loc[i, column_price] - C
            df.loc[i, column_sum_invested] = C
        
            number = []
            for j in range(0, len(amounts_S)):
                if j==0:
                    number.append(amounts_S[j]/p0)
                else:
                    number.append(amounts_S[j]/(p0*prod(j, procent)))
            k0 = number[0]  
            K = k0
            S0 = k0*p0
            C = S0  
            t = 0
            
        elif df[column_price][i] < p_buy:  
            t = t + 1
            
            if t < len(amounts_S):            
                k0 = number[t]          
                K = K + k0
                p0 = df[column_price][i]
                S0 = k0 * p0
                C = C + S0
                
                p_sell = (C/K)*(1 + r_fin/100)
                
                if (t + 1) < len(amounts_S):
                    p_buy = p0 * (1 - procent[1])
                else:
                    p_buy = p0 * (1 - procent_loss/100)

                df.loc[i, column_p_sell] = p_sell
                df.loc[i, column_p_buy] = p_buy
                df.loc[i, column_profit] = Profit
                df.loc[i, column_sell_buy] = 'buy'
                df.loc[i, column_count_buy] = k0
                df.loc[i, column_count_total_buy] = K
                df.loc[i, column_costs_of_bying] = S0
                df.loc[i, column_sum_invested] = C
                
            elif t == len(amounts_S):
                count_step[t] = count_step[t] + 1
                count_step
                df.loc[i, column_day_profit] = K * df.loc[i, column_price] - C
                size_profit[t] = size_profit[t] + K*df[column_price][i] - C  
                df.loc[i, column_sell_buy] = 'StopLoss'
                df.loc[i, column_count_sell] = K
                df.loc[i, column_sum_invested] = C

                Profit = Profit + K*df[column_price][i] - C
                df.loc[i, column_profit] = Profit
                
                p0 = df[column_price][i]
                p_sell = p0 * (1 + r/100)
                p_buy = p0 * (1 - procent[1])

                df.loc[i, column_p_sell] = p_sell
                df.loc[i, column_p_buy] = p_buy
                
                number = []
                for j in range(0, len(amounts_S)):
                    if j==0:
                        number.append(amounts_S[j]/p0)
                    else:
                        number.append(amounts_S[j]/(p0*prod(j, procent)))
#                 print('Stoploss', K * df[column_price][i] - C)
                k0 = number[0]
                K = k0
                C = k0 * p0
                t = 0
        else:        
            df.loc[i, column_p_sell] = df.loc[i-1, column_p_sell]
            df.loc[i, column_p_buy] = df.loc[i-1, column_p_buy]
            df.loc[i, column_profit] = Profit
            df.loc[i, column_sum_invested] = C
            df.loc[i, column_sell_buy] = 'waiting'
            
    df['day_profit'] = round(df['day_profit'], 0)
    df['total_profit'] = round(df['total_profit'], 0)
#     df['Date'] = pd.to_datetime(df['Date'], format="%Y/%m/%d")
    df['p_buy'] = round(df['p_buy'], 2)
    df['p_sell'] = round(df['p_sell'], 2)
    df['count_buy'] = round(df['count_buy'],2)
    df['count_sell'] = round(df['count_sell'],2)
    df['count_total_buy'] = round(df['count_total_buy'],2)
    df['costs_of_bying'] = round(df['costs_of_bying'], 2)
    df['sum_invested'] = round(df['sum_invested'], 2)


    return df   

In [20]:
work_share = all_share_in_table

In [21]:
# work_share = ['BCH',
#  'BTC',
#  'ETH',
#  'LTC',
#  'XRP',
#  'DASH',
#  'XMR']

In [22]:
work_share

['BCH',
 'BTC',
 'ETH',
 'LTC',
 'XRP',
 'DASH',
 'ETC',
 'XMR',
 'BTC_ETH',
 'BTC_ETC',
 'ETH_ETC']

In [23]:
df['Open'][0]

399.0

In [24]:
start = timer()
for share in work_share:
    model_5k(df, share_dict_start_end[share][0], share_dict_start_end[share][1])
duration = timer() - start
print(duration)

NameError: name 'column_price' is not defined

In [ ]:
df.head()

In [ ]:
df.to_csv('Result_Coins_2020_02_16_Art.csv', sep = ';', index=False)